In [1]:
import numpy as np
import pandas as pd
# importing packages. See https://github.com/BasisResearch/collab-creatures for repo setup
import logging
import os
import random
import time
import warnings

import dill
import matplotlib.pyplot as plt

warnings.simplefilter(action="ignore", category=FutureWarning)

from collab.utils import find_repo_root

root = find_repo_root()
from collab.foraging import random_hungry_followers as rhf
from collab.foraging.toolkit.visibility import construct_visibility

from collab.foraging.toolkit.utils import generate_grid
from collab.foraging.toolkit.velocity import add_velocities_to_data_object

logging.basicConfig(format="%(message)s", level=logging.INFO)

# users can ignore smoke_test -- it's for automatic testing on GitHub, to make sure the notebook runs on future updates to the repository
smoke_test = "CI" in os.environ
num_frames = 5 if smoke_test else 50
num_svi_iters = 10 if smoke_test else 1000
num_samples = 10 if smoke_test else 1000


notebook_starts = time.time()

#from .trace import rewards_trace
#from .utils import generate_grid

In [2]:
random_foragers_sim = rhf.RandomForagers(
    grid_size=40,
    probabilities=[1, 2, 3, 2, 1, 2, 3, 2, 1],
    num_foragers=3,
    num_frames=num_frames,
    num_rewards=15,
    grab_range=3,
)

random_foragers_sim()

sim = random_foragers_sim

grid = generate_grid(sim.grid_size)

grid = grid.sample(frac=1, random_state=42)

sim.grid = grid

vis = construct_visibility(
        sim.foragers,
        sim.grid_size,
        visibility_range=10,
        time_shift=0,
        grid=grid,
    )
sim.visibility_range = 10
sim.visibility = vis["visibility"]
sim.visibilityDF = vis["visibilityDF"]

add_velocities_to_data_object(sim)

/home/rafal/s78projects/collaborative-intelligence/collab/foraging/random_hungry_followers/random_foragers.py:163: UserWarning: Warning: forager movements truncated to grid size. Try running again, or increase grid size.
  warnings.warn(


In [10]:
# for b in range(1, sim.num_foragers + 1):

time_shift = 0
finders_tolerance = 2
info_time_decay = 3

visibility_restriction = "visible"


def filter_by_visibility(
    sim, 
    subject: int, 
    time_shift: int,
    visibility_restriction: str, 
    info_time_decay: int = 1,
    finders_tolerance: float = 2.0, 
    filter_by_on_reward: bool = False
):

    other_foragers_df = sim.foragersDF[sim.foragersDF["forager"] != subject]

    myself = sim.foragersDF[sim.foragersDF["forager"] == subject]

    filtered_foragers = []
    for t in range(time_shift + 1, (time_shift + len(sim.foragers[0]))):
        x_series = myself["x"][myself["time"] == t]
        y_series = myself["y"][myself["time"] == t]


        if isinstance(x_series, pd.Series) and len(x_series) == 1:
            myself_x = x_series.item()
            myself_y = y_series.item()
        else:
            myself_x = x_series
            myself_y = y_series

        others_now = other_foragers_df[other_foragers_df["time"] == t].copy()
        
        others_now["distance"] = np.sqrt(
            (others_now["x"] - myself_x) ** 2 + (others_now["y"] - myself_y) ** 2
        )

        others_now["out_of_range"] = others_now["distance"] > sim.visibility_range

        if visibility_restriction == "invisible":
            others_now = others_now[others_now["out_of_range"]]
        elif visibility_restriction == "visible":
            others_now = others_now[~others_now["out_of_range"]]

        if filter_by_on_reward:
            on_reward = []
            for _, row in others_now.iterrows():
                others_x = row["x"]
                others_y = row["y"]

                on_reward.append(
                    any(
                        np.sqrt(
                            (others_x - sim.rewards[t - time_shift - 1]["x"]) ** 2
                            + (others_y - sim.rewards[t - time_shift - 1]["y"]) ** 2
                        )
                        <= finders_tolerance
                    )
                )

            others_now["on_reward"] = on_reward

        filtered_foragers.append(others_now)
    filtered_foragersDF = pd.concat(filtered_foragers)

    if filter_by_on_reward:
        filtered_foragersDF = filtered_foragersDF[
            filtered_foragersDF["on_reward"]
            == True  # noqa E712   the == True  removal or switching to `is True` leads to failure
        ]

    expansion = [
        filtered_foragersDF.assign(time=filtered_foragersDF["time"] + i)
        for i in range(1, info_time_decay + 1)
    ]

    expansion_df = pd.concat(expansion, ignore_index=True)

    revelant_othersDF = pd.concat([filtered_foragersDF, expansion_df])


    return revelant_othersDF


filter_by_visibility(sim, subject = 1, time_shift = 0, 
                    visibility_restriction = "visible",
                    info_time_decay = 1, finders_tolerance = 1, filter_by_on_reward = False)

,x,y,time,forager,type,velocity_x,velocity_y,distance,out_of_range
0,21.0,19.0,2,2,random,0.0,0.0,3.162278,False
0,23.0,16.0,2,3,random,0.0,0.0,5.385165,False
1,22.0,23.0,3,2,random,1.0,4.0,8.944272,False
1,22.0,18.0,3,3,random,-1.0,2.0,5.000000,False
2,26.0,26.0,4,2,random,4.0,3.0,9.848858,False
2,19.0,21.0,4,3,random,-3.0,3.0,5.000000,False
3,23.0,18.0,5,3,random,4.0,-3.0,3.605551,False
9,20.0,22.0,11,3,random,-2.0,-4.0,8.062258,False
12,25.0,21.0,14,3,random,2.0,-3.0,5.656854,False
13,22.0,25.0,15,3,random,-3.0,4.0,8.062258,False


,x,y,time,forager,type,velocity_x,velocity_y,distance,out_of_range
0,21.0,19.0,1,2,random,0.0,0.0,3.162278,False
0,23.0,16.0,1,3,random,0.0,0.0,5.385165,False
1,22.0,23.0,2,2,random,1.0,4.0,8.944272,False
1,22.0,18.0,2,3,random,-1.0,2.0,5.000000,False
2,26.0,26.0,3,2,random,4.0,3.0,9.848858,False
2,19.0,21.0,3,3,random,-3.0,3.0,5.000000,False
3,23.0,18.0,4,3,random,4.0,-3.0,3.605551,False
9,20.0,22.0,10,3,random,-2.0,-4.0,8.062258,False
12,25.0,21.0,13,3,random,2.0,-3.0,5.656854,False
13,22.0,25.0,14,3,random,-3.0,4.0,8.062258,False
